In [1]:
# ftle_blue_small.py
import numpy as np
import matplotlib.pyplot as plt
import os

# === CONFIGURATION ===
T = 10
h = 0.01
frame_dt = 0.1
periods = 30
omega = (6 * np.pi) / 10
A = 0.1
epsilon = 0.25
blob_radius = 0.002  # very small blob
num_particles = 500

# === FUNCTIONS ===
def a(t): return epsilon * np.sin(omega * t)
def b(t): return 1 - 2 * epsilon * np.sin(omega * t)
def f(x, t): return a(t) * x**2 + b(t) * x
def df_dx(x, t): return 2 * a(t) * x + b(t)

def compute_flow(X, Y, t):
    fx = f(X, t)
    dfx = df_dx(X, t)
    x_dot = -np.pi * A * np.sin(np.pi * fx) * np.cos(np.pi * Y)
    y_dot = np.pi * A * np.cos(np.pi * fx) * np.sin(np.pi * Y) * dfx
    return x_dot, y_dot

def rk4_flow(X, Y, t, h):
    k1x, k1y = compute_flow(X, Y, t)
    k2x, k2y = compute_flow(X + 0.5*h*k1x, Y + 0.5*h*k1y, t + 0.5*h)
    k3x, k3y = compute_flow(X + 0.5*h*k2x, Y + 0.5*h*k2y, t + 0.5*h)
    k4x, k4y = compute_flow(X + h*k3x, Y + h*k3y, t + h)
    X_new = X + (h/6)*(k1x + 2*k2x + 2*k3x + k4x)
    Y_new = Y + (h/6)*(k1y + 2*k2y + 2*k3y + k4y)
    return X_new, Y_new

# === GRID SETUP ===
x_vals = np.linspace(0, 2, 300)
y_vals = np.linspace(0, 1, 150)
X0, Y0 = np.meshgrid(x_vals, y_vals)

# === OUTPUT ===
output_folder = "blue_small"
os.makedirs(output_folder, exist_ok=True)

# --- Place blob at manually specified blue FTLE coordinates ---
manual_start_x = 0.40
manual_start_y = 0.60

angles = 2*np.pi*np.random.rand(num_particles)
radii = blob_radius * np.sqrt(np.random.rand(num_particles))
blob_X = manual_start_x + radii * np.cos(angles)
blob_Y = manual_start_y + radii * np.sin(angles)

# --- Generate frames ---
t0_values = np.arange(0, periods*(10/3), frame_dt)
for frame_idx, t0 in enumerate(t0_values):
    t_max = t0 + T
    X, Y = X0.copy(), Y0.copy()
    for t in np.arange(t0, t_max+h, h):
        X, Y = rk4_flow(X, Y, t, h)

    # Compute FTLE
    dx = x_vals[1] - x_vals[0]
    dy = y_vals[1] - y_vals[0]

    J11 = (X[2:,1:-1] - X[:-2,1:-1]) / (2*dx)
    J12 = (X[1:-1,2:] - X[1:-1,:-2]) / (2*dy)
    J21 = (Y[2:,1:-1] - Y[:-2,1:-1]) / (2*dx)
    J22 = (Y[1:-1,2:] - Y[1:-1,:-2]) / (2*dy)

    ftle = np.full(J11.shape, np.nan)
    for i in range(ftle.shape[0]):
        for j in range(ftle.shape[1]):
            J = np.array([[J11[i,j], J12[i,j]],
                          [J21[i,j], J22[i,j]]])
            delta = J.T @ J
            eigvals = np.linalg.eigvalsh(delta)
            max_eig = np.max(eigvals)
            if max_eig > 1e-12 and np.isfinite(max_eig):
                ftle[i,j] = (1.0/T) * np.log(np.sqrt(max_eig))

    ftle_full = np.full_like(X0, np.nan)
    ftle_full[1:-1,1:-1] = ftle

    # Advect blob
    for t in np.arange(t0, t0+frame_dt+h, h):
        blob_X, blob_Y = rk4_flow(blob_X, blob_Y, t, h)

    # Plot and save
    plt.figure(figsize=(12,6))
    plt.contourf(X0, Y0, np.ma.masked_invalid(ftle_full), levels=100, cmap='jet')
    plt.colorbar(label='FTLE')
    plt.scatter(blob_X, blob_Y, color='black', s=5)
    plt.title(f"FTLE + Blue Blob | t={t0:.1f} → {t0+T:.1f}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.xlim(0,2)
    plt.ylim(0,1)
    plt.gca().set_aspect('equal')
    filename = os.path.join(output_folder, f"ftle_blue_small_{frame_idx:04d}.png")
    plt.savefig(filename, dpi=150)
    plt.close()
    print(f"Saved: {filename}")


Saved: blue_small/ftle_blue_small_0000.png
Saved: blue_small/ftle_blue_small_0001.png
Saved: blue_small/ftle_blue_small_0002.png
Saved: blue_small/ftle_blue_small_0003.png
Saved: blue_small/ftle_blue_small_0004.png
Saved: blue_small/ftle_blue_small_0005.png
Saved: blue_small/ftle_blue_small_0006.png
Saved: blue_small/ftle_blue_small_0007.png
Saved: blue_small/ftle_blue_small_0008.png
Saved: blue_small/ftle_blue_small_0009.png
Saved: blue_small/ftle_blue_small_0010.png
Saved: blue_small/ftle_blue_small_0011.png
Saved: blue_small/ftle_blue_small_0012.png
Saved: blue_small/ftle_blue_small_0013.png
Saved: blue_small/ftle_blue_small_0014.png
Saved: blue_small/ftle_blue_small_0015.png
Saved: blue_small/ftle_blue_small_0016.png
Saved: blue_small/ftle_blue_small_0017.png
Saved: blue_small/ftle_blue_small_0018.png
Saved: blue_small/ftle_blue_small_0019.png
Saved: blue_small/ftle_blue_small_0020.png
Saved: blue_small/ftle_blue_small_0021.png
Saved: blue_small/ftle_blue_small_0022.png
Saved: blue

In [2]:
# make_blue_small_movie.py
import cv2
import os

frame_folder = "blue_small"  # folder with the frames
output_video = "ftle_blue_small_movie.mp4"
fps = 20

# Get all frame files
frames = sorted([f for f in os.listdir(frame_folder) if f.endswith(".png")])

# Read first frame to get size
first_frame = cv2.imread(os.path.join(frame_folder, frames[0]))
height, width, _ = first_frame.shape

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Add frames
for f in frames:
    frame_path = os.path.join(frame_folder, f)
    img = cv2.imread(frame_path)
    video.write(img)
    print(f"Added frame: {f}")

video.release()
print(f"Movie saved: {output_video}")


Added frame: ftle_blue_small_0000.png
Added frame: ftle_blue_small_0001.png
Added frame: ftle_blue_small_0002.png
Added frame: ftle_blue_small_0003.png
Added frame: ftle_blue_small_0004.png
Added frame: ftle_blue_small_0005.png
Added frame: ftle_blue_small_0006.png
Added frame: ftle_blue_small_0007.png
Added frame: ftle_blue_small_0008.png
Added frame: ftle_blue_small_0009.png
Added frame: ftle_blue_small_0010.png
Added frame: ftle_blue_small_0011.png
Added frame: ftle_blue_small_0012.png
Added frame: ftle_blue_small_0013.png
Added frame: ftle_blue_small_0014.png
Added frame: ftle_blue_small_0015.png
Added frame: ftle_blue_small_0016.png
Added frame: ftle_blue_small_0017.png
Added frame: ftle_blue_small_0018.png
Added frame: ftle_blue_small_0019.png
Added frame: ftle_blue_small_0020.png
Added frame: ftle_blue_small_0021.png
Added frame: ftle_blue_small_0022.png
Added frame: ftle_blue_small_0023.png
Added frame: ftle_blue_small_0024.png
Added frame: ftle_blue_small_0025.png
Added frame: